**About** : This notebook is used to train models.

In [ ]:
# %load_ext nb_black
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src/

## Initialization

### Imports

In [ ]:
import os
import torch

print(torch.__version__)
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
device = torch.cuda.get_device_name(0)
print(device)

In [ ]:
import os
import sys
import glob
import json
import cudf
import torch
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.metrics import *

pd.set_option('display.width', 500)
pd.set_option('max_colwidth', 100)

In [ ]:
from params import *
from utils.plots import *
from data.preparation import *

## Data preparation

In [ ]:
df = prepare_data(DATA_PATH)

In [ ]:
df.head()

In [ ]:
def process_sign_2(path, remove_face=True):
    sign = cudf.read_parquet(path, columns=['frame', 'type', 'landmark_index', 'x', 'y', 'z'])
    
    for k in FACE_LANDMARKS.keys():
        sign.loc[(sign['landmark_index'].isin(FACE_LANDMARKS[k])) & (sign['type'] == "face"), 'type'] = k
        
    sign.loc[(sign['landmark_index'].isin(ARM_LANDMARKS)) & (sign['type'] == "pose"), 'type'] = "arm"
    
    if remove_face:
        sign = sign[sign['type'] != "face"].reset_index(drop=True)
        sign = sign[sign['type'] != "pose"].reset_index(drop=True)

    sign['type'] = sign['type'].map(TYPE_MAPPING)

    sign[['x', 'y', 'z']].fillna(-10, inplace=True)
    assert not sign.isna().sum().max()

    dfg = sign[["frame", "type", "x", "y", "z"]].astype('float32').groupby('frame').agg(list)
    return np.array(dfg.to_pandas().values.tolist()), sign

In [ ]:
def filter_sign(sign):
    frames_to_remove = sign[sign["type"].isin(['right_hand', "left_hand"])].copy()
    frames_to_remove['x'] = frames_to_remove['x'].isna()
    frames_to_remove = frames_to_remove[["frame", "x"]].groupby(['frame']).mean()

    frames_to_remove = frames_to_remove[frames_to_remove['x'] == 1].index.values.tolist()

    sign = sign[~sign['frame'].isin(frames_to_remove)]
    
    n_frames = len(sign) // 543
    divisor = (n_frames - 50) // 50 + 1
    if divisor:
        sign = sign[(sign['frame'] % divisor) == 0]
        
    return sign

In [ ]:
def process_sign_3(path, remove_face=True):
    sign = cudf.read_parquet(path, columns=['frame', 'type', 'landmark_index', 'x', 'y', 'z'])
    
    if len(sign) // 543 > 50:  # Remove no hands & use stride
        sign = filter_sign(sign)
    
    # Remove useless landmarks
    for k in FACE_LANDMARKS.keys():
        sign.loc[(sign['landmark_index'].isin(FACE_LANDMARKS[k])) & (sign['type'] == "face"), 'type'] = k
    sign.loc[(sign['landmark_index'].isin(ARM_LANDMARKS)) & (sign['type'] == "pose"), 'type'] = "arm"
    
    if remove_face:
        sign = sign[sign['type'] != "face"].reset_index(drop=True)
        sign = sign[sign['type'] != "pose"].reset_index(drop=True)

    # Map type & fill nans
    sign['type'] = sign['type'].map(TYPE_MAPPING)

    sign[['x', 'y', 'z']].fillna(-10, inplace=True)
    assert not sign.isna().sum().max()

    # Group by frame
    dfg = sign[["frame", "type", "x", "y", "z"]].astype('float32').groupby('frame').agg(list)
    return np.array(dfg.to_pandas().values.tolist()), sign

In [ ]:
save_folder = '../input/processed_3/'
os.makedirs(save_folder, exist_ok=True)
    
for i in tqdm(range(len(df))):
    path = df['path'][i]
    name = f"{path.split('/')[-2]}_{path.split('/')[-1].split('.')[0]}.npy"
    
#     if os.path.exists(save_folder + name):
#         continue

    x, sign = process_sign_3(path, remove_face=True)
    np.save(save_folder + name, x)
    
#     break

In [ ]:
%%time
x, sign = process_sign_3(df["path"][0], remove_face=True)

In [ ]:
x.shape

### Plot

In [ ]:
idx = 0
for idx in range(100):
    sign = cudf.read_parquet(df['path'][idx])
    sign['y'] *= -1

    frames_to_remove = sign[sign["type"].isin(['right_hand', "left_hand"])]
    frames_to_remove['x'] = frames_to_remove['x'].isna()
    frames_to_remove = frames_to_remove[["frame", "x"]].groupby(['frame']).mean()
    
    frames_to_remove = frames_to_remove[frames_to_remove['x'] == 1].index.values.tolist()

    sign = sign[~sign['frame'].isin(frames_to_remove)]

    n_frames = len(sign) // 543
    
    if n_frames > 50:
        print(idx, n_frames)
#     break
        

In [ ]:
idx = 331  # 45
sign = pd.read_parquet(df['path'][idx])
sign['y'] *= -1

frames_to_remove = sign[sign["type"].isin(['right_hand', "left_hand"])].copy()
frames_to_remove['x'] = frames_to_remove['x'].isna()
frames_to_remove = frames_to_remove[["frame", "x"]].groupby(['frame']).mean()

frames_to_remove = frames_to_remove[frames_to_remove['x'] == 1].index.values.tolist()

sign = sign[~sign['frame'].isin(frames_to_remove)]

In [ ]:
n_frames_before = len(sign)  // 543

# n_frames = 500
divisor = (n_frames - 50) // 50 + 1
if divisor:
    sign = sign[(sign['frame'] % divisor) == 0]
    
n_frames_after = len(sign)  // 543

print(f'Frames {n_frames_before} -> {n_frames_after}')

In [ ]:
animate(sign, label=df['sign'][idx])

Done ! 